# GStar Assignment Problem 2: Triton Weighted Row Sum

This notebook implements **Stage 2** of the GStar bootcamp: writing your first GPU kernel with weighted row sums using Triton.

## Problem Description

Write a Triton kernel to compute the weighted sum of each row in a matrix:

```
Y[i] = sum_{j=0}^{N_COLS-1} X[i, j] * W[j]
```

This introduces key concepts:

- Parallel thinking and memory access patterns
- Block-based efficiency in GPU kernels
- Triton JIT compilation and optimization


## Section 1: Import Required Libraries

Import torch, triton, and other necessary libraries for implementing the weighted row sum kernel.


In [ ]:
import torch
import triton
import triton.language as tl
import torch.nn.functional as F
import math
import argparse
import sys
import time

# Check if CUDA is available
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"CUDA capability: {torch.cuda.get_device_capability()}")

## Section 2: Implement Weighted Row Sum Kernel

Complete the weighted_row_sum_kernel function using Triton JIT compilation, including memory access patterns, block-based computation, and the forward pass function.


In [ ]:
@triton.jit
def weighted_row_sum_kernel(
    X_ptr,        # Pointer to the input tensor
    W_ptr,        # Pointer to the weight vector
    Y_ptr,        # Pointer to the output vector
    N_COLS,       # Number of columns in the input tensor
    BLOCK_SIZE: tl.constexpr  # Block size for the kernel
):
    """
    Triton kernel to compute the weighted sum of each row in a matrix.
    Y[i] = sum_{j=0}^{N_COLS-1} X[i, j] * W[j]
    """
    # 1. Get the row index for the current program instance.
    #    Hint: Use tl.program_id(axis=0).
    row_idx = tl.program_id(axis=0)

    # 2. Create a pointer to the start of the current row in the input tensor X.
    #    Hint: The offset depends on the row index and the number of columns (N_COLS).
    row_start_ptr = X_ptr + row_idx * N_COLS
    
    # 3. Create a pointer for the output vector Y.
    output_ptr = Y_ptr + row_idx

    # 4. Initialize an accumulator for the sum of the products for a block.
    #    This should be a block-sized tensor of zeros.
    #    Hint: Use tl.zeros with shape (BLOCK_SIZE,) and dtype tl.float32.
    accumulator = tl.zeros((BLOCK_SIZE,), dtype=tl.float32)

    # 5. Iterate over the columns of the row in blocks of BLOCK_SIZE.
    #    Hint: Use a for loop with tl.cdiv(N_COLS, BLOCK_SIZE).
    for col_block_start in range(0, tl.cdiv(N_COLS, BLOCK_SIZE) * BLOCK_SIZE, BLOCK_SIZE):
        # - Calculate the offsets for the current block of columns.
        #   Hint: Start from the block's beginning and add tl.arange(0, BLOCK_SIZE).
        col_offsets = col_block_start + tl.arange(0, BLOCK_SIZE)
        
        # - Create a mask to prevent out-of-bounds memory access for the last block.
        #   Hint: Compare col_offsets with N_COLS.
        mask = col_offsets < N_COLS
        
        # - Load a block of data from X and W safely using the mask.
        #   Hint: Use tl.load with the appropriate pointers, offsets, and mask.
        #   Use `other=0.0` to handle out-of-bounds elements.
        x_chunk = tl.load(row_start_ptr + col_offsets, mask=mask, other=0.0)
        w_chunk = tl.load(W_ptr + col_offsets, mask=mask, other=0.0)
        
        # - Compute the element-wise product and add it to the accumulator.
        accumulator += x_chunk * w_chunk
        
    # 6. Reduce the block-sized accumulator to a single scalar value after the loop.
    #    Hint: Use tl.sum().
    final_sum = tl.sum(accumulator)

    # 7. Store the final accumulated sum to the output tensor Y.
    #    Hint: Use tl.store().
    tl.store(output_ptr, final_sum)
    
# --- END OF STUDENT IMPLEMENTATION ---


def weighted_row_sum_forward(x: torch.Tensor, w: torch.Tensor) -> torch.Tensor:
    """
    Forward pass for the weighted row-sum operation using the Triton kernel.
    """
    assert x.dim() == 2, "Input tensor must be a 2D matrix"
    assert w.dim() == 1, "Weight tensor must be a 1D vector"
    assert x.shape[1] == w.shape[0], "Inner dimensions must match"
    assert x.is_cuda and w.is_cuda, "Tensors must be on CUDA"
    
    N_ROWS, N_COLS = x.shape
    
    # The output is a 1D tensor with length equal to the number of rows.
    y = torch.empty(N_ROWS, device=x.device, dtype=torch.float32)
    
    # The grid is 1D, with one program instance per row.
    grid = (N_ROWS,)
    
    # Block size is a power of 2. 1024 is a good default.
    BLOCK_SIZE = 1024
    
    # Launch the kernel
    weighted_row_sum_kernel[grid](
        x, w, y,
        N_COLS=N_COLS,
        BLOCK_SIZE=BLOCK_SIZE
    )
    
    return y.to(x.dtype) # Cast back to original dtype

def torch_weighted_row_sum(x: torch.Tensor, w: torch.Tensor) -> torch.Tensor:
    """
    Reference implementation using pure PyTorch.
    """
    return (x * w).sum(dim=1)

print("✅ Weighted row sum kernel implementation completed!")

## Section 3: Test Implementation with Autograder

Run the autograder code to validate correctness and performance of the weighted row sum implementation against reference implementations.


In [ ]:
# Set up data type for testing
DTYPE = torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16

def repeat_kv(x, num_groups):
    """Helper function to repeat K/V heads for GQA naive implementation."""
    if num_groups == 1:
        return x
    B, H_kv, N, D = x.shape
    x = x.unsqueeze(2).expand(B, H_kv, num_groups, N, D)
    return x.reshape(B, H_kv * num_groups, N, D)

def create_mask_bool(
    seq_len: int,
    window_size: int,
    sink_size: int,
    device=None
    ) -> torch.Tensor:
    
    idx = torch.arange(seq_len, device=device)
    row = idx.unsqueeze(1)   # (seq_len, 1)
    col = idx.unsqueeze(0)   # (1, seq_len)

    # 1) sliding window:  i - (window_size-1) <= j <= i
    sliding = (col <= row) & (col >= row - (window_size - 1))

    # 2) sink at start:   j < sink_size  *and*  j <= i
    sink = (col < sink_size) & (col <= row)

    return sliding | sink

def naive_attention(Q, K, V, is_causal=False, window_size=None, sink_size=None):
    """
    A correct, robust PyTorch implementation of standard attention for comparison.
    Supports GQA, Sliding Window, and Attention Sinks.
    """
    
    batch_size, num_heads_q, seq_len, head_dim = Q.shape
    _, num_heads_kv, seq_len, head_dim = K.shape

    if num_heads_q != num_heads_kv:
        num_groups = num_heads_q // num_heads_kv
        K = repeat_kv(K, num_groups)
        V = repeat_kv(V, num_groups)

    scale = 1.0 / math.sqrt(head_dim)
    scores = torch.matmul(Q, K.transpose(-2, -1)) * scale

    if window_size is not None:
        assert sink_size is not None, "Must specify sink_size when using window_size"
        mask_bool = create_mask_bool(seq_len, window_size, sink_size, device=scores.device)
        scores = scores.masked_fill(~mask_bool.unsqueeze(0).unsqueeze(0), float('-inf'))
    elif is_causal:
        mask = torch.tril(torch.ones(seq_len, seq_len, device=scores.device, dtype=torch.bool))
        scores = scores.masked_fill(~mask.unsqueeze(0).unsqueeze(0), float('-inf'))

    L = torch.logsumexp(scores, dim=-1, keepdim=False)
    P = torch.softmax(scores, dim=-1)
    
    O = torch.matmul(P, V)
    return O, L

def check_problem_1():
    """Checks Problem 1: PyTorch Tiled Attention."""
    problem_num = 1
    print(f"\n--- Running Autograder for Problem {problem_num}: Tiled Flash Attention ---")
    try:
        # Note: For this notebook, we're only testing problem 2, so we'll skip problem 1
        print("Problem 1 implementation not included in this notebook. Skipping...")
        return
    except ImportError:
        print(f"Could not import FlashAttention2Function from solution_{problem_num}.py.")
        return

def check_problem_2():
    """Checks Problem 2: Triton Weighted Row-Sum."""
    problem_num = 2
    print(f"\n--- Running Autograder for Problem {problem_num}: Triton Weighted Row-Sum ---")
        
    torch.manual_seed(43)
    test_cases = [(512, 1024), (1024, 4096), (2048, 8192), (4096, 8192)]
    
    def run_p2_test(rows, cols):
        x = torch.randn(rows, cols, device='cuda', dtype=DTYPE)
        w = torch.randn(cols, device='cuda', dtype=DTYPE)
        torch_result = torch_weighted_row_sum(x, w)
        triton_result = weighted_row_sum_forward(x, w)
        
        param_str = f"(Rows={rows}, Cols={cols})"
        if torch.allclose(torch_result, triton_result, rtol=5e-2, atol=5e-2):
            print(f"✅ P{problem_num} Correctness Test Passed! {param_str}")
            return True
        else:
            print(f"❌ P{problem_num} Correctness Test Failed! {param_str}")
            print(f" Max diff: {(torch_result - triton_result).abs().max()}")
            return False
            
    results = [run_p2_test(*case) for case in test_cases]
    if all(results): 
        print(f"\nAll P{problem_num} correctness tests passed!")
    else:
        print(f"\nSome P{problem_num} tests failed. Please check your implementation.")

# Run the autograder for Problem 2
check_problem_2()

## Test Your Implementation

Run a quick test to see your Triton kernel in action:


In [ ]:
# Quick functionality test
print("\n--- Quick Functionality Test ---")
torch.manual_seed(42)

# Create test tensors
rows, cols = 4, 8
x = torch.randn(rows, cols, device='cuda', dtype=torch.float32)
w = torch.randn(cols, device='cuda', dtype=torch.float32)

print(f"Input matrix X shape: {x.shape}")
print(f"Weight vector W shape: {w.shape}")
print(f"\nX =\n{x}")
print(f"\nW = {w}")

# Compute results
torch_result = torch_weighted_row_sum(x, w)
triton_result = weighted_row_sum_forward(x, w)

print(f"\nPyTorch result: {torch_result}")
print(f"Triton result:  {triton_result}")
print(f"Max difference: {torch.abs(torch_result - triton_result).max().item():.2e}")
print(f"Results match: {torch.allclose(torch_result, triton_result, rtol=1e-5, atol=1e-5)}")

## Summary

🎉 **Congratulations!** You've successfully implemented your first GPU kernel using Triton!

### Key Concepts Learned:

1. **Parallel Programming**: Each program instance handles one row independently
2. **Memory Access Patterns**: Efficient block-based loading with proper masking
3. **Triton JIT Compilation**: Writing GPU kernels in Python with automatic optimization
4. **Block-based Computation**: Processing data in chunks for better memory locality

### What's Next:

- **Problem 3**: Implement Flash Attention forward pass using Triton
- **Problem 4**: Add causal masking for language model applications
- **Advanced Problems**: Grouped Query Attention, Sliding Window, and more!

This foundation in parallel GPU computing will be essential for implementing the more complex Flash Attention algorithms in the upcoming stages.
